## **bronze-companies-income-statement**
<p>This notebook ingests unprocessed income statement data such as revenue, gross profit, net income, and EPS. The data forms a foundational layer for further financial modeling and company performance tracking.</p>

#### **Importing necessary libraries**

In [1]:
import yfinance as yf
import pandas as pd
import json
import datetime

StatementMeta(, 9890c0d1-af78-4b22-b173-b60e76967a1f, 5, Finished, Available, Finished)

#### **Variables**

In [ ]:
file_name = "comp_incm_stmt_data"
layer_name = "bronze"

#### **Step 1: Load the company codes (tickers) from the CSV**

In [2]:
codes = spark.read.csv('Files/master_files/company_code.csv', header=True)
short_codes = [
    row['short_code'].strip()
    for row in codes.select('short_code').distinct().collect()
    if row['short_code']
]

StatementMeta(, 9890c0d1-af78-4b22-b173-b60e76967a1f, 6, Finished, Available, Finished)

#### **Step 2: Fetch Income Statement for each ticker**

In [3]:
output = []

for i in short_codes:
    try:
        tickers = yf.Ticker(i)
        incm = tickers.income_stmt

        if not incm.empty:
            incm['Company'] = i

            incm.reset_index(inplace=True)
            incm.rename(columns = {'index' : 'Breakdown'}, inplace=True)
            final_incm = incm.melt(id_vars = ['Breakdown','Company'],var_name='Date', value_name='Value')

            output.append(final_incm)
    except Exception as e:
        print(f"An error occurred for {i}: {e}")



StatementMeta(, 9890c0d1-af78-4b22-b173-b60e76967a1f, 7, Finished, Available, Finished)

#### **Step 3: Combine all DataFrames and save for Bronze layer**

In [4]:
if output:
    final_incm_df = pd.concat(output, ignore_index=True) 
    
    spark_df = spark.createDataFrame(final_incm_df)

    spark_df.write.mode("overwrite").partitionBy('Date').parquet(f"Files/{layer_name}/{file_name}")

    print(f"[SUCCESS] Stock income statement data written to Lakehouse")
else:
    print("[WARNING] No data to write.")

StatementMeta(, 9890c0d1-af78-4b22-b173-b60e76967a1f, 8, Finished, Available, Finished)

[SUCCESS] Stock income statement data written to Lakehouse path: Files/bronze/comp_incm_stmt_data
